In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '..//src/')

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl
import seaborn as sns
import sys
import random
import geopandas as gpd
from common import enums, functions, schema, viz_functions
import main
from pathlib import Path

random.seed(42)

# blues = [x for x in reversed(sns.color_palette("Blues_d", 11))]
# sns.set_palette(sns.color_palette("Paired", 10))
# sns.set_style("white")
sns.set_palette(sns.color_palette("colorblind", 15))
plt.rcParams['font.family'] = 'Times New Roman'
font_size = 12
params = {'axes.labelsize': font_size+2,
          'axes.titlesize':font_size+4,
          'legend.fontsize': font_size, 
          'xtick.labelsize': font_size,
          'ytick.labelsize': font_size,
          'font.size': font_size}
plt.rcParams.update(params)
sns.set_style("whitegrid")
export_fig_format = "pdf"
fig_format = "png"
dpi = 300
figsize = (12, 3.5)
figsize_map = (16, 8)
sns.palplot(sns.color_palette())

markers = ['P', 'o', '^', 's', '*', 'v']
linewidth = 0.1
edgecolor = 'black'

print(os.getcwd())
computer_path= ""
for x in os.getcwd().split(os.path.sep):
    print(x)
    computer_path=computer_path+x+os.path.sep
    if x == "sceac10":
        break    
    else: computer_path = os.getcwd().split(os.path.sep)[0]+os.path.sep
print(computer_path)


In [ ]:
PATH_ORG = main.PATH_ORG
PATH_TABLES = PATH_ORG / r"General/communication/tables"
PATH_SAVE_FIGURES = PATH_ORG / r"General/communication/figures"
PATH_TABLES

## Number of households - projection from FES 2023 - Consumer transformation - NZ.22-NZ.25: Estimated number of heating technologies and the associated emissions reduction to 2050

In [ ]:
fn = "Dwellings_size.csv"
dataf = pd.read_csv(PATH_TABLES/fn, index_col=0, thousands=r',')
dataf = dataf.dropna(how='any').T
dataf = dataf/1000000
dataf.head()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
dataf.plot(ax=ax)
ax.set_ylabel('Number of households in millions')
ax.set_xlabel('Year')
ax.legend().remove()
ax.margins(0, None)

fig.savefig(PATH_SAVE_FIGURES/f"dwelling_projections.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"dwelling_projections.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

## Normalised profile for equipment gains

In [7]:
PATH_EQUIPMENT_GAINS_PROFILE = PATH_ORG /  r'General/resources/Equipment_gain_profile_TM59.xlsx'
SHEET_EQUIPMENT_GAINS_PROFILE = 'Normalised'

equipment_gains_df = pd.read_excel(PATH_EQUIPMENT_GAINS_PROFILE, sheet_name=SHEET_EQUIPMENT_GAINS_PROFILE, index_col=0)

In [ ]:



fig, ax = plt.subplots(figsize=figsize)
equipment_gains_df.loc['Normalised profile', :].plot(ax=ax)
ax.set_ylabel('Normalised hourly\nequipment gains profile')
ax.set_xlabel('Hours of the day')
ax.legend().remove()
ax.margins(0, None)
ax.set_ylim(0, None)

fig.savefig(PATH_SAVE_FIGURES/f"equipment_gains.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"equipment_gains.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

## Temperature profiles from CIBSE

In [ ]:
column_names = [
  'Year', 'Month', 'Day', 'Hour', 'PWC', 'Cloud', 'DBT', 'WBT', 'RH', 'Press', 'WD', 'WS', 'GSR', 'DSR', 'Alt', 'Dec', 'Cloud1', 'DBT1', 'WBT1', 'Press1', 'WD1', 'WS1'
]
pathlist = Path(main.PATH_CIBSE_DATA).rglob('*_DSY2_2050High50*.csv')

dict_cibse_temperature: dict[str, pd.Series] = {}
for path in pathlist:
  print(path.stem)
  if ('London' not in path.stem) and ('Belfast' not in path.stem):
    fig, ax = plt.subplots(figsize=figsize)
    la_str = path.stem.split('_DSY2_2050High50')[0]
    temp_dataf = pd.read_csv(path,
                             skiprows=32,
                             header=None,
                             delimiter=",",
                             names=column_names)
    weather_data = functions.format_weather_data(temp_dataf)
    weather_data.resample('1w').agg({schema.DataSchema.SOLARRADIATION:'sum', schema.DataSchema.OAT:'mean'}).plot.scatter(
        x=schema.DataSchema.SOLARRADIATION, y=schema.DataSchema.OAT, ax=ax)

In [ ]:
column_names = [
  'Year', 'Month', 'Day', 'Hour', 'PWC', 'Cloud', 'DBT', 'WBT', 'RH', 'Press', 'WD', 'WS', 'GSR', 'DSR', 'Alt', 'Dec', 'Cloud1', 'DBT1', 'WBT1', 'Press1', 'WD1', 'WS1'
]
pathlist = Path(main.PATH_CIBSE_DATA).rglob('*_DSY2_2050High50*.csv')

dict_cibse_temperature: dict[str, pd.Series] = {}
for path in pathlist:
  print(path.stem)
  if ('London' not in path.stem) and ('Belfast' not in path.stem):
    la_str = path.stem.split('_DSY2_2050High50')[0]
    temp_dataf = pd.read_csv(path, skiprows=32, header=None, delimiter=",", names=column_names, parse_dates=True)
    dict_cibse_temperature[la_str] = functions.format_weather_data(temp_dataf)[schema.DataSchema.OAT]

In [ ]:
cibse_temp_df = pd.DataFrame(dict_cibse_temperature)
cibse_temp_df
fig, ax = plt.subplots(figsize=figsize)
for c in cibse_temp_df:
  cibse_temp_df[c].sort_values(ascending=False).reset_index(drop=True).plot(
      ax=ax)
ax.margins(0, None)
ax.set_ylabel(schema.VisualisationSchema.HOURLY_OAT)
ax.set_xlabel("Number of hours")
ax.legend(bbox_to_anchor=(0.8, -0.2), ncol=4, borderaxespad=0.)
fig.savefig(PATH_SAVE_FIGURES/f"ldc_temperature_cibse.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"ldc_temperature_cibse.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)


In [ ]:
fig, ax = plt.subplots(figsize=figsize)
cibse_temp_df = pd.DataFrame(dict_cibse_temperature)
cibse_temp_df.groupby([cibse_temp_df.index.hour]).mean().plot(ax=ax, linewidth=0.8)
ax.set_ylabel(schema.VisualisationSchema.HOURLY_OAT)
ax.set_xlabel("")
ax.legend(bbox_to_anchor=(0.8, -0.13), ncol=4, borderaxespad=0.)
ax.margins(0, None)
fig.savefig(PATH_SAVE_FIGURES/f"average_hourly_temperature.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"average_hourly_temperature.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)
fig

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
cibse_temp_df.plot(ax=ax, linewidth=0.8)
ax.set_ylabel(schema.VisualisationSchema.HOURLY_OAT)
ax.set_xlabel("")
ax.legend(bbox_to_anchor=(0.8, -0.13), ncol=4, borderaxespad=0.)
ax.margins(0, None)

fig.savefig(PATH_SAVE_FIGURES/f"cibse_temperature_profiles_DSY2_2050High50.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"cibse_temperature_profiles_DSY2_2050High50.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

## Cooling demand in GB

In [ ]:
residential_data = main.import_thermal_characteristics_data(
    main.PATH_GB_DATA / 'Thermal_characteristics_afterEE.csv', 2020, 2050)
residential_data = residential_data.loc[residential_data[schema.DwellingDataSchema.THERMAL_CAPACITY_LEVEL]=="medium"]
residential_data.head()

In [ ]:
import icecream as ic
PATH_RESULTS = PATH_ORG / r'General/04 - Analysis/2050 high emission medium thermal capacity'
try:
  cooling_demand = pd.read_csv(PATH_RESULTS /
                                'cooling_demand_profiles_la.csv',
                                index_col=0, parse_dates=True)
  lsoa_peak = pd.read_csv(PATH_RESULTS / 'cooling_peak_results_lsoa.csv',
                          index_col=[0, 1], parse_dates=True)
except:
  lsoa_peak, cooling_demand = functions.extract_cooling_demand_profiles_and_peaks(
      PATH_RESULTS, residential_data)

In [ ]:
pathlist = (Path(PATH_RESULTS) /
            'simulation').rglob('*_total_heating_outputs.csv')

index_demand_frames: dict[int, pd.Series] = {}
for temp_path in pathlist:
  ic.ic(temp_path)
  la_str = temp_path.stem.split('_total_heating_outputs')[0]
  la_code = la_str.split('_')[-1]
  temp_sim_results = pd.read_csv(temp_path, index_col=0, parse_dates=True)
  index_demand_frames[la_code] = -temp_sim_results.sum()

index_demand = pd.concat(index_demand_frames).to_frame()
index_demand.columns = [schema.VisualisationSchema.COOLINGDEMAND]

## Correlation between OAT and cooling demand

In [17]:
lookup_cibse_to_ladcd = residential_data[[schema.DwellingDataSchema.CIBSE_CITY, schema.DwellingDataSchema.LADCD]].drop_duplicates()

In [18]:
lookup_cibse_to_ladcd = lookup_cibse_to_ladcd.groupby(schema.DwellingDataSchema.CIBSE_CITY).agg({schema.DwellingDataSchema.LADCD:lambda x: list(x)})

In [ ]:
daily_cooling_demand = cooling_demand.resample('1d').sum()
daily_cooling_demand.to_csv(PATH_RESULTS /
                            'daily_cooling_demand_profiles_la.csv')
daily_cooling_demand.head()

In [ ]:
cibse_temp_df.head(500).plot()

In [21]:
cibse_daily_temp_df = cibse_temp_df.resample('1d').mean()

In [ ]:
# The daily cooling demand against OAT is not as correlated as similar graph using heat demand as solar irradiance has a big impact on the cooling demand
# A composite outside air temperature metric would need to be used to have better correlation.
for city, list_lacd in lookup_cibse_to_ladcd.head(1).iterrows():
  fig, ax = plt.subplots(figsize=figsize)
  temp_df = daily_cooling_demand.loc[:, list_lacd.values[0]]
  for c in temp_df.columns:

    y_val = temp_df[c].values
    x_val = cibse_daily_temp_df[city]
    ax.scatter(x=x_val, y=y_val)

In [ ]:
# Number of cooling degree days per city
cibse_temp_df.map(lambda x: functions.calculate_cooling_degree(x, 24)).sum()/24

### Specific cooling demand by dwelling type, dwelling category and heating systems

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
(lsoa_peak/1000).reset_index(drop=True).plot.hist(bins=20, ax=ax)
ax.set_xlabel('Peak cooling demand (MW)')
ax.legend().remove()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
index_demand.reset_index(drop=True).plot.hist(bins=10, ax=ax)



In [26]:
index_demand.index = index_demand.index.droplevel(0)
index_demand.index= index_demand.index.astype(int)

In [ ]:
all_data = pd.merge(index_demand, residential_data, left_index=True, right_index=True)
all_data[schema.ResultSchema.SPECIFICCOOLINGDEMAND_DWELLING] = all_data[schema.ResultSchema.COOLINGDEMAND]/all_data[schema.DwellingDataSchema.NB_DWELLINGS]
all_data[schema.ResultSchema.SPECIFICCOOLINGDEMAND_AREA] = all_data[
    schema.ResultSchema.SPECIFICCOOLINGDEMAND_DWELLING] / all_data[
        schema.DwellingDataSchema.FLOOR_AREA]
all_data.to_csv(PATH_RESULTS/'cooling_demand_results.csv')
all_data.head()


In [ ]:
all_data.head()
temp_pivot_data = pd.pivot_table(all_data, values='Specific cooling demand (kWh/dwelling)', columns='Dwelling forms', index='LSOA_code')
temp_pivot_data.describe()

In [ ]:
all_data.head()
temp_pivot_data = pd.pivot_table(all_data, values='Average floor area m2', columns='Dwelling forms', index='LSOA_code')
temp_pivot_data.describe()

In [ ]:
all_data.head()
temp_pivot_data = pd.pivot_table(all_data, values='Specific cooling demand (kWh/m2)', columns='Dwelling forms', index='LSOA_code')
temp_pivot_data.describe()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
all_data[schema.DwellingDataSchema.DWELLING_FORMS] = [x.capitalize() for x in all_data[schema.DwellingDataSchema.DWELLING_FORMS]]
sns.boxplot(all_data, x=schema.DwellingDataSchema.DWELLING_FORMS, y=schema.ResultSchema.SPECIFICCOOLINGDEMAND_DWELLING, ax=ax, showfliers=False)
ax.set_ylabel(schema.VisualisationSchema.SPECIFICCOOLINGDEMAND_DWELLING)
ax.set_ylim(0, None)
ax.get_yaxis().set_major_formatter(
    tkr.FuncFormatter(lambda x, p: "{:,.0f}".format(x)))
fig.savefig(PATH_SAVE_FIGURES/f"dwelling_cooling_demand.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"dwelling_cooling_demand.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)
fig

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
sns.boxplot(all_data, x=schema.DwellingDataSchema.DWELLING_FORMS, y=schema.ResultSchema.SPECIFICCOOLINGDEMAND_AREA, ax=ax)
ax.set_ylabel(schema.VisualisationSchema.SPECIFICCOOLINGDEMAND_AREA)
ax.set_ylim(0, None)
fig.savefig(PATH_SAVE_FIGURES/f"specific_cooling_demand.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"specific_cooling_demand.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)
fig

In [ ]:
(cooling_demand.sum(axis=1)/1000000).sum()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

ax2 = ax.twinx()
(cooling_demand.sum(axis=1)/1000000).plot(ax=ax)
ax.set_ylabel('Hourly cooling demand [GW]', color=sns.color_palette()[0])
ax.set_xlabel('Time')
ax.set_ylim(0, 300)
ax.legend().remove()
ax.margins(0, None)

pd.DataFrame(dict_cibse_temperature).mean(axis=1).plot(ax=ax2, linewidth=0.5, color='black', linestyle='--')
ax2.set_ylabel(f'Average {schema.VisualisationSchema.HOURLY_OAT.lower()}')
ax2.set_ylim(0, 35)
ax2.grid(False)
# ax.set_ylim(0, None)
fig.savefig(PATH_SAVE_FIGURES/f"cooling_demand_GB.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"cooling_demand_GB.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)
fig

In [35]:
### Electricity for cooling

In [36]:
scop=14/3.412 #https://www.eia.gov/todayinenergy/detail.php?id=40232
cop = scop*0.875

In [ ]:
ic.ic('electricity for cooling', (cooling_demand.sum(axis=1)/1000000/scop).sum())
ic.ic('max cooling demand',(cooling_demand.sum(axis=1)/1000000/cop).max())
ic.ic('95% percentiles cooling demand',(cooling_demand.sum(axis=1)/1000000/cop).quantile(0.95))

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax2 = ax.twiny()
(cooling_demand.sum(axis=1) / 1000000 / cop).plot(ax=ax, label='Hourly demand')
ldc = cooling_demand.sum(axis=1).copy()
ldc = ldc.sort_values(ascending=False).reset_index(drop=True)/1000000/ cop
ldc.plot(ax=ax2, label='Load duration curve', marker='o', markevery=100, color=sns.color_palette()[1])
ax.set_ylabel('Hourly electricity\nfor cooling demand [GW]')
ax.set_xlabel('')

ax2.grid(False)
ax.xaxis.grid(False)
ax2.set_xlim(0, None)
ax2.set_xlabel('Number of hours')
ax2.xaxis.label.set_color(sns.color_palette()[1])
ax2.tick_params(axis='x', colors=sns.color_palette()[1])
ax2.get_xaxis().set_major_formatter(tkr.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.margins(0, None)
ax.set_ylim(0, None)

lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)
fig.savefig(PATH_SAVE_FIGURES/f"electricity_for_cooling_demand_GB.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"electricity_for_cooling_demand_GB.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

In [ ]:
ldc[ldc>0.1]*1000

In [ ]:
average_week_day = cooling_demand.sum(axis=1).groupby(
    [cooling_demand.index.hour]).mean().reset_index(drop=True)
average_week_day.plot()

In [ ]:
# Peak cooling day
from datetime import datetime
filt = cooling_demand.index.date == datetime(2050,7,19).date()
(cooling_demand.sum(axis=1) / 1000000 / cop).loc[filt].plot()

In [42]:
path_map = PATH_ORG /  r'General/resources/GIS map and data'
file = "Local_authority_UK_2019.geojson"
path_map = path_map/file

In [60]:
from mpl_toolkits.axes_grid1 import make_axes_locatable


def create_map_LA(df,
                  target,
                  path_map,
                  fig,
                  ax,
                  cmap=None,
                  percentage_scale=False, label_legend:str=''):

  map_df = gpd.read_file(path_map)
  map_df.crs = {'init': 'epsg:4326'}

  map_df = pd.merge(map_df,
                    df,
                    left_on="lad19cd",
                    right_on=schema.DwellingDataSchema.LADCD,
                    how="left")

  map_df = map_df.dropna(subset=[target])
  # create figure and axes for Matplotlib
  if percentage_scale:
    vmin = 0
    vmax = 1
  else:
    vmin = 0
    vmax = map_df[target].max()

  if cmap == None:
    cmap = 'Blues'

  ax.axis('off')
  ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

  map_df = map_df.to_crs({'init': 'epsg:3395'})  # mercator projections
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.1)

  format_axis = tkr.FuncFormatter(lambda x, p: "{:,.0f}".format(x))
  if map_df[target].max()<100:

    format_axis = tkr.FuncFormatter(lambda x, p: "{:,.1f}".format(x))

  map = map_df.plot(column=target,
              cmap=cmap,
              linewidth=0.015,
              ax=ax,
              edgecolor='black',
              vmin=vmin,
              vmax=vmax,
              legend=True,
              legend_kwds={
                      "label": label_legend,
                      "orientation": "vertical",
                      "shrink": .2,
                       "format": format_axis
                      },
              cax=cax)  #,
  plt.close()
  return map



In [ ]:
# Peak cooling demand in kW
la_cooling_demand = cooling_demand.max()
la_cooling_demand = la_cooling_demand.to_frame()
la_cooling_demand = la_cooling_demand.reset_index()
la_cooling_demand.columns = [
    schema.DwellingDataSchema.LADCD, schema.DwellingDataSchema.COOLING_DEMAND]
la_cooling_demand.head()

In [ ]:
la_cooling_demand.sort_values(schema.DwellingDataSchema.COOLING_DEMAND, ascending=False).describe()/1000

In [ ]:
to_plot_df = la_cooling_demand.copy()

fig, ax = plt.subplots(figsize=(figsize_map))
target = schema.DwellingDataSchema.COOLING_DEMAND
to_plot_df[target] = to_plot_df[target]/1000000 #to GW

cmap_1 = "YlGnBu"
# create_map_LA(to_plot_df, target, path_map, fig, ax, cmap=cmap_1, label_legend='Peak cooling demand (MW)')
create_map_LA(to_plot_df, target, path_map, fig, ax, cmap=cmap_1)
ax.set_title(
    f"Residential peak cooling demand\nin 2050 at local authority level (unit: GW)")

fig.savefig(PATH_SAVE_FIGURES/f"residential_peak_cooling_demand_la_map.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"residential_peak_cooling_demand_la_map.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)
fig


In [ ]:
all_data['Total Floor area'] = all_data[schema.DwellingDataSchema.NB_DWELLINGS]*all_data[schema.DwellingDataSchema.FLOOR_AREA]
la_data = all_data.groupby(schema.DwellingDataSchema.LADNM).agg({schema.DwellingDataSchema.COOLING_DEMAND:'sum',
                                                       schema.DwellingDataSchema.NB_DWELLINGS:'sum',
                                                       schema.DwellingDataSchema.FLOOR_AREA:'mean',
                                                       schema.DwellingDataSchema.HEATING_DEMAND:'sum',
                                                       'Total Floor area':'sum'})


la_data.head()

In [ ]:
la_data.sort_values(schema.DwellingDataSchema.HEATING_DEMAND, ascending=False).head(10)

In [ ]:
la_data.sort_values('Total Floor area', ascending=False).head(10)

In [ ]:
la_data.sort_values(schema.DwellingDataSchema.COOLING_DEMAND, ascending=False).head(10)

In [ ]:
# Annual cooling demand GWh
la_cooling_demand = cooling_demand.sum()
la_cooling_demand = la_cooling_demand.to_frame()
la_cooling_demand = la_cooling_demand.reset_index()
la_cooling_demand.columns = [
    schema.DwellingDataSchema.LADCD, schema.DwellingDataSchema.COOLING_DEMAND]
la_cooling_demand.head()


to_plot_df = la_cooling_demand.copy()

fig, ax = plt.subplots(figsize=(figsize_map))
target = schema.DwellingDataSchema.COOLING_DEMAND
to_plot_df[target] = to_plot_df[target]/1000000

cmap_1 = "YlGnBu"
create_map_LA(to_plot_df,
              target,
              path_map,
              fig,
              ax,
              cmap=cmap_1)
# create_map_LA(to_plot_df, target, path_map, fig, ax, cmap=cmap_1, label_legend='Cooling demand (GWh)')

ax.set_title(f"Residential cooling demand\nin 2050 at local authority level (unit: GWh)")
fig.savefig(PATH_SAVE_FIGURES/f"residential_cooling_demand_lsoa_map.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"residential_cooling_demand_lsoa_map.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

fig

## LSOA map

In [69]:
pathlist = (Path(PATH_RESULTS)/'simulation').rglob('*_total_heating_outputs.csv')

lsoa_to_index_dict = dict(residential_data.reset_index().groupby('LSOA_code')['Index'].apply(list))

try:
  lsoa_cooling_demand = pd.read_csv(PATH_RESULTS / 'cooling_demand_lsoa.csv', index_col=0)
except:

  frames:dict[str, pd.Series] = {}
  for temp_path in pathlist:
    temp_sim_results = pd.read_csv(temp_path, index_col=0, parse_dates=True)
    temp_sim_results.columns = [int(x) for x in temp_sim_results.columns]
    for lsoa_code, list_index in lsoa_to_index_dict.items():
      list_index_in_file = [
          x for x in temp_sim_results.columns if x in list_index
      ]
      if len(list_index_in_file) > 0:
        ic.ic(lsoa_code)
        frames[lsoa_code] = -temp_sim_results[list_index].sum(axis=1)

  lsoa_cooling_demand = pd.concat(frames, axis=1)
  lsoa_cooling_demand.to_csv(PATH_RESULTS / 'cooling_demand_lsoa.csv')

In [71]:
map_df = viz_functions.get_UK_LSOA_map()
summary_lsoa_cooling_peak = lsoa_peak.copy()
summary_lsoa_cooling_peak = summary_lsoa_cooling_peak.droplevel(0)
# summary_lsoa_cooling_peak.index = summary_lsoa_cooling_peak.index.droplevel(0)
summary_lsoa_cooling_peak = summary_lsoa_cooling_peak/1000
summary_lsoa_cooling_peak.columns = ['Peak cooling demand (MW)']
map_df = pd.merge(map_df,
                  summary_lsoa_cooling_peak,
                  left_on='LSOA11CD',
                  right_index=True)


In [ ]:
lsoa_peak
fig, ax = plt.subplots(figsize=(figsize_map))
cmap_1 = "YlGnBu"
target = 'Peak cooling demand (MW)'
map_obj = viz_functions.plot_map(map_df,
                                 target,
                                 ax,
                                 vmin=0,
                                 cmap=cmap_1,
                                 legend=True)

ax.set_title(
    f"Residential peak cooling demand\nin 2050 at LSOA level (unit: MW)")
fig.savefig(PATH_SAVE_FIGURES/f"residential_peak_cooling_demand_lsoa_map.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"residential_peak_cooling_demand_lsoa_map.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

fig

In [ ]:
map_df = viz_functions.get_UK_LSOA_map()
summary_lsoa_cooling_demand = (lsoa_cooling_demand.sum()/1_000_000).to_frame()
summary_lsoa_cooling_demand.columns = ['Cooling demand (GWh)']
map_df = pd.merge(map_df,
                  summary_lsoa_cooling_demand,
                  left_on='LSOA11CD',
                  right_index=True)
summary_lsoa_cooling_demand.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(figsize_map))
cmap_1 = "YlGnBu"
target = f'Cooling demand (GWh)'
map_obj = viz_functions.plot_map(map_df,
                                 target,
                                 ax,
                                 vmin=0,
                                 cmap=cmap_1,
                                 legend=True)

ax.set_title(f"Residential cooling demand\nin 2050 at LSOA level (unit: GWh)")
fig.savefig(PATH_SAVE_FIGURES/f"residential_cooling_demand_lsoa_map.{fig_format}", bbox_inches='tight', format=fig_format, dpi=dpi)
fig.savefig(PATH_SAVE_FIGURES/f"residential_cooling_demand_lsoa_map.{export_fig_format}", bbox_inches='tight', format=export_fig_format, dpi=dpi)

fig